# Contents


# Climate model simulations from CMIP5
 The Model Intercomparison Projects (MIP) are standard sets of concerted experiments in which different modelling groups produce  simulations that are compared with each others and assessed against available observational data sets. The Climate Model Intercomparison Project (CMIP) was promoted by the World Climate Research Program (WCRP) to support the Intergovernmental Panel on Climate Change (IPCC) during the assessment reports (AR) and provide numerical projections of future global climate under various scenarios. It is now in phase 6 (CMIP6). CMIP1 and 2 were voluntary and not much coordinated. CMIP3 was the firs organized and concerted phase. The successive phase was renamed CMIP5 to synchronize the number with the cycle of the IPCC assessment reports. 
They are composed of a minimum set of simulations (tiers): 
* A control simulation, in which the climate model are forced with pre-industrial conditions (a constant atmospheric CO2 and other greenhouse gases). These simulations are very long (> 500 years), to produce the baseline climate. They are essential to estimate climate variability for detection and attribution studies.
* A 1% atmospheric CO$_2$ increase up to 2x the pre-industrial concentration and then stabilizes. This is a standard benchmark for models to estimate the "climate sensitivity", which is measured as the change in the mean Earth temperature for a doubling of CO2.
* An "historical" simulation (ofetn referred to as transient), where observed trajectories of GHGs and other radiatively active forcings are prescribed in the atmosphere. Other drivers such as volcanic eruptions are included in the annual changes of the chemical concentrations.
* A set of future projections, which are narratives of what the future atmospheric concentration will be based on a combination of socio-economical conditions that have been translated into concentration of active radiative forcings (called Reference Concentration Pathways since CMIP5).

Additional experiments have been done to assess the role of the various forcings, so it is possible to find historical simulations in which only the GHG concentrations have been prescribed and the ozone stays at the preindustrial level. In others, the solar activity can be prescribed as the only source of (astronomical) natural variability. 

Several CMIP data can be obtained from the Earth System Federation Grid website, which is an international consortium of IT experts and climate scientists specifically created to access and process data at peta and exa scales. It is composed of different nodes in different parts of the world, and one example is the one at the Lawrence Livermore Lab in the US:
https://esgf-node.llnl.gov/projects/esgf-llnl/

Data are organized per projects, and within each project there are different experiments with many variables. For certain experiments, there can also be ensemble simulations. An ensemble is one possible climate realization which is constrained by the same boundary conditions but evolved differently mostly because of different initial conditions (like a perturbed ocean or atmosphere state). 

There are millions of data available, and the majority of climate studies makes use of a large number of different simulations, to include as many information as possible to constrain the uncertainties linked to assessing future conditions. We will only analyse a few model outputs that have already been pre-processed. If you download raw data from ESGF you will have to run a few steps to get them ready for analysis. Different climate models (or Earth System Model, ESM, in case they include a carbon and other biogeochemical components) are identified by an institute name and by a model name. One institute can have different models, for instance with different horizontal resolutions or different components. 

4 models have been provided, with climatological data for the historical period 1979-2005, which coincides with the satellite data we analysed previously. Monthly and daily climatology have been computed from the orginal daily data.
* CMCC (Centro EuroMediterraneo per i Cambiamenti Climatici, Italy): CMCC-CM
* NOAA-GFDL (National Oceanic and Atmospheric Administration - Geophysical Fluid Dynamics Lab, USA): GFDL-CM3
* CCCMA (Canadian Centre for Climate Modelling and Analysis, Canada) CanESM
* CSIRO (Commonwealth Scientific and Industrial Research Organisation, Australia) ACCESS1-0

Data are available on the course website in NetCDF format. There is one TGZ file for each collection of monthly, daily and grid area data.